# Анализ рекомендательных систем на данных Retail Rocket E-commerce Dataset
Задание: На данных https://www.kaggle.com/datasets/retailrocket/ecommerce-dataset/data применить алгоритмы ALS, BPR (как в общем примере по MovieLens), сравнить оценки.

Для начала загрузим и подготовим данные из Retail Rocket E-commerce Dataset:

In [ ]:
!pip install implicit watermark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import random
from scipy.sparse import coo_matrix, csr_matrix
import implicit
from sklearn.model_selection import train_test_split
from IPython.display import HTML
from watermark import watermark
import time

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

HTML("""
<style>
    .dataframe thead tr:only-child th { text-align: right; }
    .dataframe thead th { text-align: left; padding: 5px; }
    .dataframe tbody tr th { vertical-align: top; padding: 5px; }
    .dataframe tbody tr:hover { background-color: #ffff99; }
    .dataframe { background-color: white; color: black; font-size: 16px; }
</style>
""")

print(watermark(python=True, watermark=True, iversions=True, globals_=globals()))


Python implementation: CPython
Python version       : 3.11.12
IPython version      : 7.34.0

IPython  : 7.34.0
numpy    : 2.0.2
implicit : 0.7.2
pandas   : 2.2.2
scipy    : 1.15.3
watermark: 2.5.0
sklearn  : 1.6.1

Watermark: 2.5.0



In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("retailrocket/ecommerce-dataset")

print("Path to dataset files:", path)

100%|██████████| 291M/291M [00:07<00:00, 42.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/retailrocket/ecommerce-dataset/versions/2


Загрузка и предварительная обработка данных

In [ ]:
events = pd.read_csv(path+"/events.csv")

# оставим только взаимодействия, которые можно считать implicit feedback (например, 'view')
events = events[events['event'] == 'view']

# закодируем id-шники
user_map = {id_: i for i, id_ in enumerate(events['visitorid'].unique())}
item_map = {id_: i for i, id_ in enumerate(events['itemid'].unique())}
events['user_id'] = events['visitorid'].map(user_map)
events['item_id'] = events['itemid'].map(item_map)

# аггрегируем количество просмотров (view count)
interaction_matrix = events.groupby(['user_id', 'item_id']).size().reset_index(name='strength')

# создание разреженной матрицы
rows = interaction_matrix['user_id'].values # индексы пользователей
cols = interaction_matrix['item_id'].values  # индексы товаров
data = interaction_matrix['strength'].values # сила взаимодействия

R = coo_matrix((data, (rows, cols))).tocsr()
R

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 2132127 stored elements and shape (1404179, 234838)>

In [ ]:
print(R)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 2132127 stored elements and shape (1404179, 234838)>
  Coords	Values
  (0, 0)	1
  (0, 3341)	1
  (1, 1)	1
  (1, 2055)	1
  (1, 2369)	1
  (1, 4040)	1
  (1, 5366)	1
  (1, 8117)	1
  (1, 11346)	1
  (1, 15350)	1
  (1, 22403)	1
  (1, 24578)	2
  (1, 24622)	4
  (1, 31136)	1
  (1, 36779)	1
  (1, 37373)	1
  (1, 37867)	1
  (1, 37873)	1
  (1, 38146)	1
  (1, 38655)	2
  (1, 38843)	1
  (1, 50667)	2
  (1, 69196)	1
  (1, 73719)	1
  (1, 91657)	1
  :	:
  (1404154, 48455)	1
  (1404155, 34544)	1
  (1404156, 38908)	1
  (1404157, 1651)	1
  (1404158, 234835)	1
  (1404159, 17329)	1
  (1404160, 29802)	1
  (1404161, 12384)	1
  (1404162, 2304)	1
  (1404163, 7614)	1
  (1404164, 166485)	1
  (1404165, 234836)	1
  (1404166, 2517)	1
  (1404167, 234837)	1
  (1404168, 3912)	1
  (1404169, 32380)	1
  (1404170, 26446)	1
  (1404171, 9081)	1
  (1404172, 131681)	1
  (1404173, 47133)	1
  (1404174, 11719)	1
  (1404175, 64354)	1
  (1404176, 6799)	1
  (1404177, 8845)	1
  (

In [ ]:
from sklearn.model_selection import train_test_split
from scipy.sparse import lil_matrix

# создаём список взаимодействий
user_ids = interaction_matrix['user_id'].values
item_ids = interaction_matrix['item_id'].values
strengths = interaction_matrix['strength'].values

# разбиваем на train и test по взаимодействиям
X_train, X_test = train_test_split(
    list(zip(user_ids, item_ids, strengths)),
    test_size=0.2,
    random_state=SEED
)

# создаём пустые разреженные матрицы в формате LIL (удобно для заполнения)
train_matrix = lil_matrix(R.shape)
test_matrix = lil_matrix(R.shape)

# заполняем матрицы
for u, i, s in X_train:
    train_matrix[u, i] = s
for u, i, s in X_test:
    test_matrix[u, i] = s

# переводим обратно в CSR формат
train_matrix = train_matrix.tocsr()
test_matrix = test_matrix.tocsr()


## Обучение ALS (Alternating Least Squares)

 — метод factorization:

`R ≈ U × V^T`

где U — матрица пользователей (user factors) [n_users × factors]

V — матрица товаров (item factors) [n_items × factors]

Результат — приближённое восстановление предпочтений и возможность рекомендовать новые товары.

In [ ]:
# ALS
als_model = implicit.als.AlternatingLeastSquares(
    factors=50, regularization=0.1, iterations=30)

start_time = time.time()
als_model.fit(train_matrix)
als_time = time.time() - start_time

als_recommendations = als_model.recommend_all(train_matrix)

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
als_recommendations

array([[ 8197,  2517,  2718, ...,  4765,  2067,  2162],
       [  564,  1568,  5791, ...,  4511,  2276, 10120],
       [    9,     8,     7, ...,     2,     1,     0],
       ...,
       [82774,  3526,   586, ..., 19245,   425, 12415],
       [ 5871,  3091,   253, ..., 11526,  6099,   649],
       [  777,  4711,  5261, ...,  6418,  9986,   564]], dtype=int32)

In [ ]:
# als_recommendations: (n_users × N) — список из N item_id, рекомендованных пользователю.
df_recs = pd.DataFrame(als_recommendations)
df_recs.columns = [f'top{i+1}' for i in range(df_recs.shape[1])]
df_recs['user_id'] = df_recs.index

df_recs = df_recs[['user_id'] + [f'top{i+1}' for i in range(df_recs.shape[1] - 1)]]

In [ ]:
print('время, затраченное на обучение als:', als_time, 'cекунд')

время, затраченное на обучение als: 220.1773498058319 cекунд


 каждому пользователю ALS рекомендует топ-N товаров:

In [ ]:
df_recs

,user_id,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10
0,0,8197,2517,2718,25327,8379,745,1443,4765,2067,2162
1,1,564,1568,5791,683,224,425,52603,4511,2276,10120
2,2,9,8,7,6,5,4,3,2,1,0
3,3,586,564,1411,148,2276,9074,164,32910,921,1515
4,4,9,8,7,6,5,4,3,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...
1404174,1404174,31742,1563,2259,3637,14082,4962,20848,683,53266,9231
1404175,1404175,4543,56245,164,32380,2488,3073,55906,7035,2064,1262
1404176,1404176,82774,3526,586,6054,8959,1616,24616,19245,425,12415
1404177,1404177,5871,3091,253,3526,38843,2304,1515,11526,6099,649


## Обучение BPR (Bayesian Personalized Ranking)

— метод обучения рекомендаций по неявной обратной связи (implicit feedback), оптимизирующий ранжирование товаров, а не приближение рейтингов как ALS.

BPR обучается так, чтобы u предпочитал i перед j:

`score(u, i) > score(u, j)`

((пользователь u, товар i, с которым он взаимодействовал (например, просмотрел, кликнул), товар j, с которым он не взаимодействовал))

In [ ]:
# BPR
bpr_model = implicit.bpr.BayesianPersonalizedRanking(
    factors=50, regularization=0.1, iterations=30)

start_time = time.time()
bpr_model.fit(train_matrix)
bpr_time = time.time() - start_time

bpr_recommendations = bpr_model.recommend_all(train_matrix)

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
print('время, затраченное на обучение bpr:', bpr_time, 'cекунд')

время, затраченное на обучение bpr: 24.371084928512573 cекунд


In [ ]:
bpr_recommendations

array([[110252,  10318,  25753, ...,  66224, 133222,  27325],
       [110252,  10318,  25753, ...,  66224, 133222,  27325],
       [110252,  10318,  25753, ...,  66224, 133222,  27325],
       ...,
       [110252,  10318,  25753, ...,  66224, 133222,  27325],
       [110252,  10318,  25753, ...,  66224, 133222,  27325],
       [110252,  10318,  25753, ...,  66224, 133222,  27325]], dtype=int32)

In [ ]:
df_bpr = pd.DataFrame(bpr_recommendations)
df_bpr.columns = [f'top{i+1}' for i in range(df_bpr.shape[1])]
df_bpr['user_id'] = df_bpr.index

df_bpr = df_bpr[['user_id'] + [f'top{i+1}' for i in range(df_bpr.shape[1] - 1)]]
df_bpr

,user_id,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10
0,0,110252,10318,25753,61374,12361,25068,143285,66224,133222,27325
1,1,110252,10318,25753,61374,12361,25068,143285,66224,133222,27325
2,2,110252,10318,25753,61374,12361,25068,143285,66224,133222,27325
3,3,110252,10318,25753,61374,12361,25068,143285,66224,133222,27325
4,4,110252,10318,25753,61374,12361,25068,143285,66224,133222,27325
...,...,...,...,...,...,...,...,...,...,...,...
1404174,1404174,110252,10318,25753,61374,12361,25068,143285,66224,133222,27325
1404175,1404175,110252,10318,25753,61374,12361,25068,143285,66224,133222,27325
1404176,1404176,110252,10318,25753,61374,12361,25068,143285,66224,133222,27325
1404177,1404177,110252,10318,25753,61374,12361,25068,143285,66224,133222,27325


итог и цель обучения: модель учится ранжировать товары так, чтобы заимодействованные стояли выше, чем невзаимодействованные.



Сравним ALS и BPR:

In [ ]:
df_compare = df_recs.merge(df_bpr, on='user_id', suffixes=('_als', '_bpr'))
df_compare.head()

,user_id,top1_als,top2_als,top3_als,top4_als,top5_als,top6_als,top7_als,top8_als,top9_als,...,top1_bpr,top2_bpr,top3_bpr,top4_bpr,top5_bpr,top6_bpr,top7_bpr,top8_bpr,top9_bpr,top10_bpr
0,0,8197,2517,2718,25327,8379,745,1443,4765,2067,...,110252,10318,25753,61374,12361,25068,143285,66224,133222,27325
1,1,564,1568,5791,683,224,425,52603,4511,2276,...,110252,10318,25753,61374,12361,25068,143285,66224,133222,27325
2,2,9,8,7,6,5,4,3,2,1,...,110252,10318,25753,61374,12361,25068,143285,66224,133222,27325
3,3,586,564,1411,148,2276,9074,164,32910,921,...,110252,10318,25753,61374,12361,25068,143285,66224,133222,27325
4,4,9,8,7,6,5,4,3,2,1,...,110252,10318,25753,61374,12361,25068,143285,66224,133222,27325


## Оценка моделей

In [ ]:
from scipy.sparse import csr_matrix

train_matrix = train_matrix.tocsr()
test_matrix = test_matrix.tocsr()

In [ ]:
import numpy as np

def precision_recall_at_k(true_matrix, pred_matrix, k=10):
    true_items = true_matrix.tolil().rows
    pred_items = np.argsort(-pred_matrix, axis=1)[:, :k]

    precisions = []
    recalls = []

    for user_id in range(len(true_items)):
        true_set = set(true_items[user_id])
        pred_set = set(pred_items[user_id])

        if len(true_set) > 0:
            precision = len(true_set & pred_set) / k
            recall = len(true_set & pred_set) / len(true_set)
            precisions.append(precision)
            recalls.append(recall)

    return np.mean(precisions), np.mean(recalls)

# предсказанные рейтинги всех пользователей ко всем товарам
als_pred_matrix = als_model.user_factors @ als_model.item_factors.T
bpr_pred_matrix = bpr_model.user_factors @ bpr_model.item_factors.T

als_precision, als_recall = precision_recall_at_k(test_matrix, als_pred_matrix, k=10)
bpr_precision, bpr_recall = precision_recall_at_k(test_matrix, bpr_pred_matrix, k=10)

print(f"ALS Precision@10: {als_precision:.2f}")
print(f"ALS Recall@10: {als_recall:.2f}")
print()
print(f"BPR Precision@10: {bpr_precision:.2f}")
print(f"BPR Recall@10: {bpr_recall:.2f}")

ALS Precision@10: 0.03
ALS Recall@10: 0.07

BPR Precision@10: 0.05
BPR Recall@10: 0.10
